---
# 3.0 Interpolation and Polynomial Approximation 
---

A function $P$ is a **polynomial of degree at most $n$** if

$$P(x) = a_0 + a_1 x + \cdots + a_n x^n.$$

Polynomials hold a prominent role in numerical analysis for three reasons:

1. Polynomials can approximate any continuous function $f$ as close as we want:

 > ### [Weierstrass Approximation Theorem](http://en.wikipedia.org/wiki/Stone%E2%80%93Weierstrass_theorem):
 >
 > Let $f \in C[a,b]$. For every $\varepsilon > 0$, there exists a polynomial $P(x)$ such that
 >
 > $$\left|f(x) - P(x)\right| < \varepsilon, \quad \forall x \in [a,b].$$
 
![title](Fig3.1.png)

2. Polynomials can be efficiently evaluated using **Horner's Rule**:

  $$P(x) = \bigg( \Big( \big(a_n x + a_{n-1}\big)x + a_{n-2} \Big)x + \cdots + a_1\bigg)x + a_0$$

3. Polynomials are **simple**:  it is easy to sum, multiply, differentiate, and integrate polynomials.

---

## Horner's Rule

In [1]:
# Evaluate the polynomial 
#
#     P(x) = a_0 + a_1 x + ... + a_n x^n
#
# where a = [a_0, a_1, ..., a_n] using Horner's Rule.
function horner(x, a)
    
    n = length(a) - 1
    p = a[n+1]
    for j = n:-1:1
        p = p*x + a[j]
    end
    
    return p
end

horner (generic function with 1 method)

In [2]:
using PyPlot

a = [-3, 5, 2, -1]   # P(x) = -3 + 5x + 2x^2 - x^3

4-element Array{Int64,1}:
 -3
  5
  2
 -1

In [3]:
horner(-2., a)

3.0

In [4]:
x = -2.
p = -3 + 5x + 2x^2 - x^3

3.0

In [5]:
x = -2
@evalpoly(x, -3, 5, 2, -1)

3

---

## Two types of problems

The function $f$ we would like to approximate by a polynomial may be given to us as:

1. **A fixed set of data points**: $\big\{(x_i, y_i)\big\}_{i=0}^n$, where $y_i = f(x_i)$, but the actual function $f$ is unknown to us.
   
2. **An explicit/implicit function**: We are free to choose the $x_i$ and compute $y_i = f(x_i)$, but evaluating $f$ may be expensive.

In either case, the goal is to find a polynomial $p$ that **interpolates** the data:
   
   $$p(x_i) = y_i, \quad i = 0,1,\ldots,n.$$ 
   

---

## Estimating the function $f$

After constructing an interpolating polynomial $p$, we can use $p$ to **estimate** the value of $f$ at other values of $x$. We hope that

$$ p(x) \approx f(x), \quad \forall x \in [a,b].$$

We call the estimation of $f(x)$:

1. **interpolation** if 
    $$x_i < x < x_j, \quad \text{for some } i \neq j,$$

2. **extrapolation** if 
    $$ x < x_i,\ \forall i \quad \text{or} \quad x > x_i,\ \forall i.$$

---

## Interpolating polynomial always exists and is unique

> ### Theorem:
>
> Let $\big\{(x_i,y_i)\big\}_{i=0}^n$. If $x_i \neq x_j$ for $i \neq j$, then there exists a unique polynomial $p(x)$ with degree at most $n$ that satisfies
>
>   $$p(x_i) = y_i, \quad i = 0,1,\ldots,n.$$ 

---

## The space of polynomials $
%%% My LaTeX definitions
\DeclareMathOperator{\span}{span}
\newcommand{\Pbf}{\mathbf{P}}
$

Let $\Pbf_n$ be the set of polynomials with degree at most $n$. 

$\Pbf_n$ is a **vector space** since it is closed under addition and scalar multiplication:

1. $p_1(x), p_2(x) \in \Pbf_n \implies p_1(x)+p_2(x) \in \Pbf_n$
2. $c \in \mathbb{R}, p(x) \in \Pbf_n \implies cp(x) \in \Pbf_n$

Note that $\dim \Pbf_n = n+1$.

## A basis for $\Pbf_n$

Let $\big\{\phi_j(x)\big\}_{j=0}^n$ be a **basis** for $\Pbf_n$; that is:

1. $\phi_0(x), \ldots, \phi_n(x)$ are **linearly independent**:

    $$
    c_0 \phi_0(x) + \cdots + c_n \phi_n(x) = 0
    \quad \implies \quad
    c_0 = \cdots = c_n = 0
    $$
    
2. $\phi_0(x), \ldots, \phi_n(x)$ **spans** $\Pbf_n$:
    
    $$
    \mathbf{P}_n = \span\big\{\phi_0(x),\ldots,\phi_n(x)\big\}
    $$
    

Every $p(x) \in \Pbf_n$ is therefore a _unique_ linear combination of the polynomials in $\big\{\phi_j(x)\big\}_{j=0}^n$:
    
$$p(x) = \sum_{j=0}^n c_j \phi_j(x) = c_0 \phi_0(x) + \cdots + c_n \phi_n(x).$$

## Computing the unique interpolating polynomial 

Given $\big\{(x_i, y_i)\big\}_{i=0}^n$, we want to find the unique $p(x) \in \Pbf_n$ that satisfies

$$p(x_i) = y_i, \quad i = 0, 1, \ldots, n.$$

Thus, we just need to find scalars $c_0,\ldots,c_n$ such that

$$
\begin{gather}
c_0 \phi_0(x_0) + \cdots + c_n \phi_n(x_0) = y_0\\
c_0 \phi_0(x_1) + \cdots + c_n \phi_n(x_1) = y_1\\
\vdots\\
c_0 \phi_0(x_n) + \cdots + c_n \phi_n(x_n) = y_n\\
\end{gather}
$$

This is equivalent to the linear system $A c = y$:

$$
\begin{bmatrix}
\phi_0(x_0) & \phi_1(x_0) & \cdots & \phi_n(x_0)\\
\phi_0(x_1) & \phi_1(x_1) & \cdots & \phi_n(x_1)\\
\vdots & \vdots & \ddots & \vdots\\
\phi_0(x_n) & \phi_1(x_n) & \cdots & \phi_n(x_n)\\
\end{bmatrix}
\begin{bmatrix}
c_0\\
c_1\\
\vdots\\
c_n\\
\end{bmatrix}
=
\begin{bmatrix}
y_0\\
y_1\\
\vdots\\
y_n\\
\end{bmatrix}.
$$

## Using different bases for $\Pbf_n$

Any basis $\big\{\phi_j(x)\big\}_{j=0}^n$ we use will give us the same interpolating polynomial $p(x)$, but different bases will have different computational properties.

We will study **three** different bases:

1. (Section 3.0) **Monomial basis** $\{1, x, x^2, \ldots, x^n\}$, i.e., $\phi_i=x^i$ ($i=0,1,\ldots,n$), for which the matrix $A$ is often ill-conditioned;
2. (Section 3.1) **Lagrange polynomial basis**, for which the matrix $A$ is the identity matrix $I$;
3. (Section 3.2-3.4) **Newton polynomial basis**, for which the matrix $A$ is lower triangular.

In each case we will look at how to **construct** $p(x)$ and how to **evaluate** $p(x)$.

---

---

## 3.0 Monomial interpolation

---

$%%% My LaTeX definitions
\DeclareMathOperator{\span}{span}
\newcommand{\Pbf}{\mathbf{P}}
$
When using the **monomial basis** $\{1, x, x^2, \ldots, x^n\}$ to find the polynomial $p(x) \in \Pbf_n$ that interpolates the data point $\{(x_i,y_i)\}_{i=0}^n$, the matrix $A$ is given by

$$
A = 
\begin{bmatrix}
1 & x_0 & \cdots & x_0^n\\
1 & x_1 & \cdots & x_1^n\\
\vdots & \vdots & \ddots & \vdots\\
1 & x_n & \cdots & x_n^n\\
\end{bmatrix}.
$$

This is a [Vandermonde matrix](http://en.wikipedia.org/wiki/Vandermonde_matrix). 

## Determinant of the Vandermonde matrix

We can compute the determinant of $A$ as follows (using $n=3$ for simplicity).

First we do some row-reduction which does not change the determinant, i.e., $L_i-L_1\to L_i$ ( $i=2,\ldots,4$ ).

$$
\begin{align}
\det(A)
& = 
\begin{vmatrix}
1 & x_0 & x_0^2 & x_0^3\\
1 & x_1 & x_1^2 & x_1^3\\
1 & x_2 & x_2^2 & x_2^3\\
1 & x_3 & x_3^2 & x_3^3\\
\end{vmatrix} \\ \\
& = 
\begin{vmatrix}
1 & x_0 & x_0^2 & x_0^3\\
0 & x_1-x_0 & x_1^2-x_0^2 & x_1^3-x_0^3\\
0 & x_2-x_0 & x_2^2-x_0^2 & x_2^3-x_0^3\\
0 & x_3-x_0 & x_3^2-x_0^2 & x_3^3-x_0^3\\
\end{vmatrix} \\ \\
\end{align}
$$

Then we do some column-reduction steps, which again do not change the determinant, i.e., $C_{i+1}-x_0C_{i}\to C_{i+1}$ ( $i=3,\ldots, 1$ ).

$$
\begin{align}
\det(A)
& = 
\begin{vmatrix}
1 & x_0 & x_0^2 & 0 \\
0 & x_1-x_0 & x_1^2-x_0^2 & x_1^3-x_0^3 - x_0(x_1^2-x_0^2)\\
0 & x_2-x_0 & x_2^2-x_0^2 & x_2^3-x_0^3 - x_0(x_2^2-x_0^2)\\
0 & x_3-x_0 & x_3^2-x_0^2 & x_3^3-x_0^3 - x_0(x_3^2-x_0^2)\\
\end{vmatrix} \\ \\
& = 
\begin{vmatrix}
1 & x_0 & x_0^2 & 0 \\
0 & x_1-x_0 & x_1^2-x_0^2 & (x_1 - x_0)x_1^2\\
0 & x_2-x_0 & x_2^2-x_0^2 & (x_2 - x_0)x_2^2\\
0 & x_3-x_0 & x_3^2-x_0^2 & (x_3 - x_0)x_3^2\\
\end{vmatrix} \\ \\
& = 
\begin{vmatrix}
1 & x_0 & 0 & 0 \\
0 & x_1-x_0 & (x_1 - x_0)x_1 & (x_1 - x_0)x_1^2\\
0 & x_2-x_0 & (x_2 - x_0)x_2 & (x_2 - x_0)x_2^2\\
0 & x_3-x_0 & (x_3 - x_0)x_3 & (x_3 - x_0)x_3^2\\
\end{vmatrix} \\ \\
& = 
\begin{vmatrix}
1 & 0 & 0 & 0 \\
0 & x_1-x_0 & (x_1 - x_0)x_1 & (x_1 - x_0)x_1^2\\
0 & x_2-x_0 & (x_2 - x_0)x_2 & (x_2 - x_0)x_2^2\\
0 & x_3-x_0 & (x_3 - x_0)x_3 & (x_3 - x_0)x_3^2\\
\end{vmatrix} \\ \\
\end{align}
$$

Next, we pull out the common factors in each of the last three rows.

$$
\begin{align}
\det(A)
& = 
\begin{vmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & x_1 & x_1^2\\
0 & 1 & x_2 & x_2^2\\
0 & 1 & x_3 & x_3^2\\
\end{vmatrix} (x_1-x_0)(x_2-x_0)(x_3-x_0) \\ \\
\end{align}
$$

We repeat the above process on the bottom-right $3 \times 3$ submatrix.

$$
\begin{align}
\det(A)
& = 
\begin{vmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & x_2\\
0 & 0 & 1 & x_3\\
\end{vmatrix} (x_2-x_1)(x_3-x_1)\cdot(x_1-x_0)(x_2-x_0)(x_3-x_0) \\ \\
\end{align}
$$

Repeating the above process, this time on the bottom-right $2 \times 2$ submatrix, we obtain.

$$
\begin{align}
\det(A)
& = 
\begin{vmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{vmatrix} (x_3-x_2)\cdot(x_2-x_1)(x_3-x_1)\cdot(x_1-x_0)(x_2-x_0)(x_3-x_0) \\ \\
\end{align}
$$

Therefore, 
$$\det(A) = (x_3-x_2)\cdot(x_2-x_1)(x_3-x_1)\cdot(x_1-x_0)(x_2-x_0)(x_3-x_0).$$

In general,
$$\det(A) = \prod_{0\leq i < j \leq n} (x_j - x_i).$$

(See https://proofwiki.org/wiki/Vandermonde_Determinant for the full proof.)

Thus, if $x_i \neq x_j$ for $i \neq j$, then $\det(A) \neq 0$, so $A$ is invertible. Therefore, $Ac = y$ has exactly one solution which implies that there always exists a unique interpolating polynomial when the $x_i$ are distinct.